In [13]:
# main library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline

# display thai language in colab : https://www.facebook.com/groups/colab.thailand/permalink/1421960354645985/
import matplotlib as mpl
plt.rcParams['font.family'] = 'tahoma'
!wget -q https://github.com/Phonbopit/sarabun-webfont/raw/master/fonts/thsarabunnew-webfont.ttf
mpl.font_manager.fontManager.addfont('thsarabunnew-webfont.ttf')
mpl.rc('font', family='TH Sarabun New', size=10)


# package for downloading file from Google drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# authenticate and create the PyDrive client.
# this only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [14]:
# download temp file
file_id = '16G6QX9ug3Mrgabn5AwVGRUUriQMeQygu'
f_dir = 'Group_Data_Activity_Adjust_OK.xlsx'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile(f_dir)

In [15]:
df = pd.read_excel("Group_Data_Activity_Adjust_OK.xlsx")[["STU_ID","Time_Cln","Activity","Pain","Category_Pain","Gain","Category_Gain","category"]].fillna("")

In [16]:
df.shape

(1091, 8)

In [17]:
df = df[df["Time_Cln"].str.strip() != ""]
df.head(5)

,STU_ID,Time_Cln,Activity,Pain,Category_Pain,Gain,Category_Gain,category
0,6210422033,08:30,ตื่นนอน,ไม่อยากตื่น อยากนอนต่อ,ง่วงนอน,ได้เริ่มต้นวันใหม่,เริ่มต้นวันใหม่,ตื่น
1,6210422033,08:35,อาบน้ำ,ยังอยากนอนต่อ,ง่วงนอน,สดชื่น+สะอาด,สะอาด,อาบน้ำ
2,6210422033,09:00,กินอาหาเช้า,ต้องสั่ง Delivery,เตรียมตัว,อิ่มท้อง,อิ่ม,กินข้าว
3,6210422033,10:00,เดินทางด้วย BTS ไปทำงาน,ราคาสูง,ค่าใช้จ่ายสูง,เดินทางสะดวกสบาย,สบายใจ,เดินทาง
4,6210422033,10:30,ถึงที่ทำงาน เริ่มทำงาน,,จุก,เป็นช่วงที่พร้อมทำงานที่สุด,พร้อมทำงาน,ทำงาน


In [18]:
ls_time = [i.replace(".","")[:5] for i in df["Time_Cln"].tolist()]

In [19]:
ls_time[255:260]

['09:00', '09:05', '10:00', '10:30', '13:00']

In [20]:
df["Int_Time"] = [int("".join(i.split(":"))) for i in ls_time]
df["Int_Time"][df["Int_Time"] == 0] = 2400
df["Int_Time"][df["Int_Time"] <= 300] = df["Int_Time"]+2400

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [21]:
df[df["Int_Time"] == 0]

,STU_ID,Time_Cln,Activity,Pain,Category_Pain,Gain,Category_Gain,category,Int_Time


In [22]:
df["Period_Thai_Time"] = np.where(df["Int_Time"] >= 2400,"ดึกมาก" ,
                                  np.where(df["Int_Time"] >= 2100,"ดึก" , 
                                           np.where(df["Int_Time"] >= 1800,"ค่ำ" ,
                                                    np.where(df["Int_Time"] >= 1500,"เย็น" ,
                                                             np.where(df["Int_Time"] >= 1200,"บ่าย" ,
                                                                      np.where(df["Int_Time"] >= 900,"สาย" ,"เช้า"
                         )
                         )
                         )
                         )
)
                                  
                                  )

In [23]:
df

,STU_ID,Time_Cln,Activity,Pain,Category_Pain,Gain,Category_Gain,category,Int_Time,Period_Thai_Time
0,6210422033,08:30,ตื่นนอน,ไม่อยากตื่น อยากนอนต่อ,ง่วงนอน,ได้เริ่มต้นวันใหม่,เริ่มต้นวันใหม่,ตื่น,830,เช้า
1,6210422033,08:35,อาบน้ำ,ยังอยากนอนต่อ,ง่วงนอน,สดชื่น+สะอาด,สะอาด,อาบน้ำ,835,เช้า
2,6210422033,09:00,กินอาหาเช้า,ต้องสั่ง Delivery,เตรียมตัว,อิ่มท้อง,อิ่ม,กินข้าว,900,สาย
3,6210422033,10:00,เดินทางด้วย BTS ไปทำงาน,ราคาสูง,ค่าใช้จ่ายสูง,เดินทางสะดวกสบาย,สบายใจ,เดินทาง,1000,สาย
4,6210422033,10:30,ถึงที่ทำงาน เริ่มทำงาน,,จุก,เป็นช่วงที่พร้อมทำงานที่สุด,พร้อมทำงาน,ทำงาน,1030,สาย
...,...,...,...,...,...,...,...,...,...,...
1086,6220422109,19:30,ว่ายน้ำ,เหนื่อย ใช้แรงเยอะ,เหนื่อยล้า,ได้ออกกำลังกาย ลดน้ำหนัก,หายหิว,ออกกำลังกาย,1930,ค่ำ
1087,6220422109,20:30,อาบน้ำ,หนาว ทำให้ผิวแห้ง,หนาว,สดชื่น สะอาด,สะอาด,อาบน้ำ,2030,ค่ำ
1088,6220422109,20:40,เล่น Social IG FB,มือถือแบตหมด,รถติด,ได้ความเพลิดเพลิน,เป็นที่ชื่นชอบ,เล่นโซเชียล,2040,ค่ำ
1089,6220422109,21:30,ดู Netflix,ติดซีรีย์ ใช้เวลาเยอะ,เสียเวลา,ดูหนังหลากหลายที่บ้าน,ดูแลตัวเอง,ดูหนังฟังเพลง,2130,ดึก


In [ ]:
df.to_excel("COMPLETE_FILE_VALUE_BASE.xlsx",index=False)

In [ ]:
!pip install pythainlp deepcut

In [ ]:
import re
import string
import pandas as pd 
import numpy as np
from pythainlp.tokenize import word_tokenize
from pythainlp.tokenize import sent_tokenize
from pythainlp.tag.named_entity import ThaiNameTagger
from pythainlp.tag import pos_tag_sents
import deepcut
from pythainlp.corpus.common import thai_stopwords

def clean_msg(msg):
    
    
    # ลบ text ที่อยู่ในวงเล็บ <> ทั้งหมด
    msg = re.sub(r'<.*?>','', msg)
    
    # ลบ hashtag
    msg = re.sub(r'#','',msg)
    
    # ลบ เครื่องหมายคำพูด (punctuation)
    for c in string.punctuation:
        msg = re.sub(r'\{}'.format(c),'',msg)
    
    # ลบ separator เช่น \n \t
    msg = ' '.join(msg.split())
    
    return msg

In [ ]:
ls_text = [" ".join(word_tokenize(clean_msg(i),engine="deepcut")) for i in df["Gain"].tolist()]

In [ ]:
ls_text[:5]

['ได้ เริ่มต้น วัน ใหม่',
 'สดชื่นสะอาด',
 'อิ่มท้อง',
 'เดินทาง สะดวกสบาย',
 'เป็น ช่วง ที่ พร้อม ทำ งาน ที่สุด']

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(ls_text)
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
df_tfidf = pd.DataFrame(denselist, columns=feature_names)

In [ ]:
####GAIN
df_tfidf

,10,19,25,about,activities,aligned,almost,and,anywhere,arrive,at,atmosphere,awake,back,because,bed,before,better,board,body,boost,bought,budget,burn,business,by,calories,cant,capabilities,car,check,checkin,clean,cleaned,close,coffee,colleages,comfortable,connect,convenient,...,แผนก,แพง,แมว,แย,แรง,แล,แลกเปล,และ,และสมอง,แวะ,แสกน,แสง,แสดง,แอบ,โจร,โชค,โดน,โดย,โดยสาร,โต,โบ,โรง,โล,โลก,โอกาส,ใกล,ใจ,ใช,ใน,ให,ใหม,ไกล,ไข,ได,ไป,ไฟฟ,ไม,ไร,ไว,ไหม
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.539668,0.0,0.0,0.248230,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1033,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.188593,0.0,0.0,0.0,0.0,0.0,0.0
1034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1035,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.248557,0.0,0.0,0.0,0.0,0.0,0.0
1036,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
col_cate = df_tfidf.index
ls_result = pd.DataFrame({col_cate[i]:df_tfidf[:,i] for i in range(df_tfidf.shape[1])}).idxmax('columns').values

TypeError: ignored

In [ ]:
test = df.groupby(["Int_Time","category"])[["STU_ID"]].count().reset_index().pivot(index="Int_Time",columns="category",values="STU_ID").fillna(0)

In [ ]:
rtest = test.cumsum()

In [ ]:
rtest

category,,กาแฟ,กิจวัตร,กินข้าว,ชา,ซื้อของ,ดูหนังฟังเพลง,ตื่น,ถึงจุดหมาย,ทำการบ้าน,ทำงาน,ทำอาหาร,นอน,ประชุม,พักผ่อน,ออกกำลังกาย,อาบน้ำ,อ่านหนังสือ,เดินทาง,เดิยทาง,เตรียมตัว,เริ่มงาน,เรียน,เลิกงาน,เล่นหุ้น,เล่นเกม,เล่นโซเชียล,แต่งตัว,ไปเที่ยว
Int_Time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
529,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
530,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
534,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
535,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2430,1.0,41.0,8.0,157.0,1.0,26.0,41.0,65.0,15.0,29.0,101.0,1.0,53.0,69.0,20.0,22.0,117.0,21.0,74.0,1.0,6.0,45.0,18.0,32.0,0.0,24.0,17.0,13.0,3.0
2500,1.0,41.0,8.0,157.0,1.0,26.0,41.0,65.0,15.0,29.0,101.0,1.0,63.0,69.0,20.0,22.0,117.0,21.0,74.0,1.0,6.0,45.0,18.0,32.0,1.0,24.0,17.0,13.0,3.0
2530,1.0,41.0,8.0,157.0,1.0,26.0,41.0,65.0,15.0,29.0,101.0,1.0,64.0,69.0,20.0,22.0,117.0,21.0,74.0,1.0,6.0,45.0,18.0,32.0,1.0,24.0,17.0,13.0,3.0


In [ ]:
!pip install bar_chart_race

In [ ]:
import bar_chart_race as bcr

In [ ]:
df_cv = bcr.load_dataset('covid19_tutorial')

In [ ]:
df_cv

,Belgium,China,France,Germany,Iran,Italy,Netherlands,Spain,USA,United Kingdom
date,,,,,,,,,,
2020-04-03,1143,3326,6520,1275,3294,14681,1490,11198,7418,3611
2020-04-04,1283,3330,7574,1444,3452,15362,1656,11947,8387,4320
2020-04-05,1447,3333,8093,1584,3603,15887,1771,12641,9489,4943
2020-04-06,1632,3335,8926,1810,3739,16523,1874,13341,10783,5385
2020-04-07,2035,3335,10343,2016,3872,17127,2108,14045,12798,6171
2020-04-08,2240,3337,10887,2349,3993,17669,2255,14792,14704,7111
2020-04-09,2523,3339,12228,2607,4110,18279,2403,15447,16553,7993
2020-04-10,3019,3340,13215,2767,4232,18849,2520,16081,18595,8974
2020-04-11,3346,3343,13851,2894,4357,19468,2653,16606,20471,9892


In [ ]:
bcr.bar_chart_race(df=rtest,interpolate_period=False, 
        title='Activity Cummu',sort="desc",steps_per_period=1,figsize=(6,4))